In [6]:
import tensorflow as tf
import torch
import torch.nn.functional as F

import numpy as np

print(tf.__version__)
print(torch.__version__)
print(np.__version__)

2.4.1
1.9.0+cpu
1.20.1


## Compare `binary_cross_entropy_with_logits`

In [19]:
y_true = np.random.uniform(size=[2, 128, 128])
y_pred = np.random.uniform(size=[2, 128, 128])

print(y_pred.shape)
print(y_true.shape)

(2, 128, 128)
(2, 128, 128)


In [31]:
tf_loss = tf.keras.losses.binary_crossentropy(
    y_true=y_true, y_pred=y_pred, from_logits=True,
)

print(tf_loss.shape)
print(tf.reduce_mean(tf_loss, axis=1))

(2, 128)
tf.Tensor([0.73223665 0.73234737], shape=(2,), dtype=float64)


In [25]:
torch_loss = F.binary_cross_entropy_with_logits(
    torch.from_numpy(y_pred), torch.from_numpy(y_true), reduction="none"
)

print(torch_loss.shape)
print(torch_loss.mean(2).shape)
print(torch_loss.mean(2).mean(1))

torch.Size([2, 128, 128])
torch.Size([2, 128])
tensor([0.7322, 0.7323], dtype=torch.float64)


## Compare `sigmoid_l1_loss`

In [33]:
logits = np.random.uniform(size=[2, 128, 128])
target = np.random.uniform(size=[2, 128, 128])
offset = -0.5
mask = np.random.uniform(size=[2, 128, 128])

In [36]:
def sigmoid_l1_loss(logits, target, offset=0.0, mask=None):
    logp = torch.sigmoid(logits) + offset
    loss = torch.abs(logp - target)
    if mask is not None:
        w = mask.mean(2, True).mean(1, True)
        w[w == 0] = 1
        loss = loss * (mask / w)

    loss = loss.mean(2).mean(1)
    print(loss)
    return loss

sigmoid_l1_loss(
    torch.from_numpy(logits),
    torch.from_numpy(target), offset, torch.from_numpy(mask))

logits: torch.Size([2, 128, 128])
target: torch.Size([2, 128, 128])
maks: torch.Size([2, 128, 128])
tensor([0.3996, 0.3989], dtype=torch.float64)


tensor([0.3996, 0.3989], dtype=torch.float64)

In [38]:
def sigmoid_l1_loss(logits, target, offset=0.0, mask=None):
    logp = tf.math.sigmoid(logits) + offset
    loss = tf.math.abs(logp - target)
    if mask is not None:
        w = tf.math.reduce_mean(mask, axis=2, keepdims=True)
        condition = tf.equal(w, 0)
        w = tf.where(condition, 1.0, w)
        loss = loss * (mask / w)

    loss = tf.math.reduce_mean(
        tf.math.reduce_mean(loss, axis=2),
        axis=1
    )
    return loss

tf_sig_l1_loss = sigmoid_l1_loss(logits, target, offset, mask)
print(tf_sig_l1_loss)

tf.Tensor([0.39963411 0.39889329], shape=(2,), dtype=float64)
